# Box Office - data aggregation

In [ ]:
# Import libraries
import pandas as pd
import re

In [2]:
#Load the dataset that contains the Youtube movie trailers
videos = pd.read_excel("videos_excel.xlsx")
videos

,Title,Published_date,Views,Likes,Comments,Channel_id,Channel_name
0,Marvel Studios’ Assembled: The Making of Ms. M...,2022-08-03,356975,14835.0,621.0,UCvC4D8onUfXzvjTOM-dBfEA,Marvel Entertainment
1,ALL-OUT AVENGERS #1 Trailer | Marvel Comics,2022-07-26,100369,4796.0,461.0,UCvC4D8onUfXzvjTOM-dBfEA,Marvel Entertainment
2,Marvel Studios’ Black Panther: Wakanda Forever...,2022-07-24,35367133,1409575.0,62712.0,UCvC4D8onUfXzvjTOM-dBfEA,Marvel Entertainment
3,Official Trailer | She-Hulk: Attorney at Law |...,2022-07-24,21715675,580781.0,36385.0,UCvC4D8onUfXzvjTOM-dBfEA,Marvel Entertainment
4,I Am Groot | Official Trailer | Disney+,2022-07-22,17876830,685331.0,17958.0,UCvC4D8onUfXzvjTOM-dBfEA,Marvel Entertainment
...,...,...,...,...,...,...,...
4866,Hope Springs - Official Trailer,2012-10-04,15928,108.0,6.0,UCf5CjDJvsFvtVIhkfmKAwAA,MGM
4867,21 Jump Street - Official Trailer,2012-10-04,45674,332.0,2.0,UCf5CjDJvsFvtVIhkfmKAwAA,MGM
4868,The Girl with the Dragon Tattoo - Official Tra...,2012-10-04,46922,252.0,5.0,UCf5CjDJvsFvtVIhkfmKAwAA,MGM
4869,Hot Tub Time Machine Official Trailer,2009-12-17,477623,1249.0,131.0,UCf5CjDJvsFvtVIhkfmKAwAA,MGM


In [3]:
# Load the dataset that contains the Domestic Box-office found in the web site <https://www.boxofficemojo.com>
box_office = pd.read_excel("BoxOffice_2016_2022.xlsx")
box_office

,Year,Rank,Release,Gross,Theaters,Total Gross,Release Date,Distributor,Re-Release
0,2022,1,Top Gun: Maverick,"$662,486,308",4751,"$662,486,308",2022-05-27,Paramount Pictures,NaN
1,2022,2,Doctor Strange in the Multiverse of Madness,"$411,327,261",4534,"$411,327,261",2022-05-06,Walt Disney Studios Motion Pictures,NaN
2,2022,3,Jurassic World Dominion,"$371,865,200",4697,"$371,865,200",2022-06-10,Universal Pictures,NaN
3,2022,4,The Batman,"$369,345,583",4417,"$369,345,583",2022-03-04,Warner Bros.,NaN
4,2022,5,Minions: The Rise of Gru,"$334,576,925",4427,"$334,576,925",2022-07-01,Universal Pictures,NaN
...,...,...,...,...,...,...,...,...,...
1395,2016,196,Green Room,"$3,220,371",777,"$3,220,371",2016-04-15,A24,NaN
1396,2016,197,Our Kind of Traitor,"$3,153,157",399,"$3,153,157",2016-07-01,Roadside Attractions,NaN
1397,2016,198,Compadres,"$3,127,773",368,"$3,127,773",2016-04-22,Pantelion Films,NaN
1398,2016,199,The Martian,"$3,088,310",3854,"$228,433,663",2016-10-02,Twentieth Century Fox,NaN


In [4]:
# remove duplicates (keep the oldest = last)
box_office = box_office.drop_duplicates(subset=["Release","Total Gross", "Distributor"], keep='last')

In [5]:
# remove movies whose re-release column is not null
box_office = box_office.drop(box_office[box_office["Re-Release"].notnull()].index)

In [6]:
#checking NAs
box_office.count()

Year            1214
Rank            1214
Release         1214
Gross           1214
Theaters        1214
Total Gross     1214
Release Date    1214
Distributor     1214
Re-Release         0
dtype: int64

In [7]:
#convert titles into lowercase
videos["Clean_Title"] = videos["Title"].str.lower()

In [34]:
#checking 
videos[videos["Clean_Title"].str.contains("the batman")]

,Title,Published_date,Views,Likes,Comments,Channel_id,Channel_name,Clean_Title,Movie
1347,THE BATMAN - The Bat and The Cat Trailer,2021-12-27,29704212,561027.0,29052.0,UCjmJDM5pRKbUlVIzDYYWb6g,Warner Bros. Pictures,the batmanthe bat and the cat,
1354,THE BATMAN – Main Trailer,2021-10-16,55480124,1299539.0,98599.0,UCjmJDM5pRKbUlVIzDYYWb6g,Warner Bros. Pictures,the batman – main,
1391,The Batman - DC FanDome Teaser,2020-08-23,39733529,1064351.0,107755.0,UCjmJDM5pRKbUlVIzDYYWb6g,Warner Bros. Pictures,the batmandc fandome,


In [9]:
#remove words that are not part of the movie title
#Note1: this step requires a previous analysis of word occurrences and it might change depending on the dataset.
#Note2: string patterns that include white space are replace for whitespace, to avoid joining two or more words.
videos["Clean_Title"] = videos["Clean_Title"].str.replace("^official trailer", "")
videos["Clean_Title"] = videos["Clean_Title"].str.strip() #remove whitespaces for the next steps
videos["Clean_Title"] = videos["Clean_Title"].str.replace("^official teaser", "")
videos["Clean_Title"] = videos["Clean_Title"].str.strip() #remove whitespaces for the next steps
videos["Clean_Title"] = videos["Clean_Title"].str.replace("^trailer", "")
videos["Clean_Title"] = videos["Clean_Title"].str.strip() #remove whitespaces for the next steps
videos["Clean_Title"] = videos["Clean_Title"].str.replace("^teaser", "")
videos["Clean_Title"] = videos["Clean_Title"].str.strip() #remove whitespaces for the next steps
videos["Clean_Title"] = videos["Clean_Title"].str.replace("^trailer", "")  #in case of "teaser trailer" at the beginning
videos["Clean_Title"] = videos["Clean_Title"].str.strip() #remove whitespaces for the next steps
videos["Clean_Title"] = videos["Clean_Title"].str.replace("official trailer.*", "")
videos["Clean_Title"] = videos["Clean_Title"].str.replace("trailer.*", "")
videos["Clean_Title"] = videos["Clean_Title"].str.replace("official teaser.*", "")
videos["Clean_Title"] = videos["Clean_Title"].str.replace("teaser.*", "")
videos["Clean_Title"] = videos["Clean_Title"].str.replace("20\d{2} movie", "")
videos["Clean_Title"] = videos["Clean_Title"].str.replace("\(20\d{2}\)", "")
videos["Clean_Title"] = videos["Clean_Title"].str.replace("\(\)", "")
videos["Clean_Title"] = videos["Clean_Title"].str.replace("marvel studios’", "")
videos["Clean_Title"] = videos["Clean_Title"].str.replace("marvel studios'", "")
videos["Clean_Title"] = videos["Clean_Title"].str.replace("marvel's", "")
videos["Clean_Title"] = videos["Clean_Title"].str.replace("\[hd\]", "")
videos["Clean_Title"] = videos["Clean_Title"].str.replace("\(hd\)", "")
videos["Clean_Title"] = videos["Clean_Title"].str.replace("20th century fox", "")
videos["Clean_Title"] = videos["Clean_Title"].str.replace("disney\+", "")
videos["Clean_Title"] = videos["Clean_Title"].str.replace(" mgm", " ")
videos["Clean_Title"] = videos["Clean_Title"].str.replace("fox searchlight", "")
videos["Clean_Title"] = videos["Clean_Title"].str.replace("paramount pictures", "")
videos["Clean_Title"] = videos["Clean_Title"].str.replace("in theaters.*", "")
videos["Clean_Title"] = videos["Clean_Title"].str.replace(" dvd", " ")
videos["Clean_Title"] = videos["Clean_Title"].str.replace(" blu-ray", " ")
videos["Clean_Title"] = videos["Clean_Title"].str.replace(" - ", " ")
videos["Clean_Title"] = videos["Clean_Title"].str.replace("\|", "")

C:\Users\ander\AppData\Local\Temp\ipykernel_12632\1089950240.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  videos["Clean_Title"] = videos["Clean_Title"].str.replace("^official trailer", "")
C:\Users\ander\AppData\Local\Temp\ipykernel_12632\1089950240.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  videos["Clean_Title"] = videos["Clean_Title"].str.replace("^official teaser", "")
C:\Users\ander\AppData\Local\Temp\ipykernel_12632\1089950240.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  videos["Clean_Title"] = videos["Clean_Title"].str.replace("^trailer", "")
C:\Users\ander\AppData\Local\Temp\ipykernel_12632\1089950240.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  videos["Clean_Title"] = videos["Clean_Title"].str.replace("^teaser", "")
C:\Users\ander\AppData\Local\Temp\ip

In [10]:
# removing unecessary whitespaces
videos["Clean_Title"] = videos["Clean_Title"].str.strip()

In [11]:
# checking if there is Clean_title with no names
videos[videos["Clean_Title"] == ""]

,Title,Published_date,Views,Likes,Comments,Channel_id,Channel_name,Clean_Title
848,Disney+ | Official Trailer,2019-08-23,345,42.0,3.0,UChCUEwJrrG01_L7w4Bh1xqA,Disney Entertainment Studios,


In [12]:
#Check how the Clean_title looks like after transformation
videos[["Clean_Title"]]

,Clean_Title
0,assembled: the making of ms. marvel
1,all-out avengers #1
2,black panther: wakanda forever
3,she-hulk: attorney at law
4,i am groot
...,...
4866,hope springs
4867,21 jump street
4868,the girl with the dragon tattoo
4869,hot tub time machine


In [13]:
#create a empty column to store the movie name
videos["Movie"] = ""

In [14]:
#reset the dataframe index because of dropped rows
box_office = box_office.reset_index(drop=True)
#df['new column name'] = df['column name'].apply(lambda x: 'value if condition is met' if x condition else 'value if condition is not met')

In [15]:
#set threshold to filter potential false positives
threshold = 0.8

In [16]:
#get movie name
for i in range(len(videos)):
    for ii in range(len(box_office)):
        #get movie name that completely matchs the Clean_title of YouTube videos
        if videos.loc[i, "Clean_Title"] == str(box_office.loc[ii, "Release"]).lower():
            videos.loc[i,"Movie"] = box_office.loc[ii,"Release"]
        #get movie name that matchs the Clean_title of YouTube videos above the threshold, 
        #and video is published before the movie release
        elif str(box_office.loc[ii, "Release"]).lower() in videos.loc[i, "Clean_Title"] and \
           len(str(box_office.loc[ii, "Release"]))/len(videos.loc[i, "Clean_Title"]) >= threshold and \
           box_office.loc[ii, "Release Date"] > videos.loc[i, "Published_date"]:
            if len(str(box_office.loc[ii,"Release"])) > len(str(videos.loc[i,"Movie"])):
                videos.loc[i,"Movie"] = box_office.loc[ii,"Release"]

In [17]:
#checking positive results
videos[videos["Movie"] != ""]

,Title,Published_date,Views,Likes,Comments,Channel_id,Channel_name,Clean_Title,Movie
15,Marvel Studios' Thor: Love and Thunder | Offic...,2022-05-24,53748416,1597284.0,61134.0,UCvC4D8onUfXzvjTOM-dBfEA,Marvel Entertainment,thor: love and thunder,Thor: Love and Thunder
18,Marvel Studios’ Doctor Strange in the Multiver...,2022-05-05,5776447,224724.0,NaN,UCvC4D8onUfXzvjTOM-dBfEA,Marvel Entertainment,doctor strange in the multiverse of madness f...,Doctor Strange in the Multiverse of Madness
22,Marvel Studios' Thor: Love and Thunder | Offic...,2022-04-18,44252005,1790078.0,68921.0,UCvC4D8onUfXzvjTOM-dBfEA,Marvel Entertainment,thor: love and thunder,Thor: Love and Thunder
36,Marvel Studios' Doctor Strange in the Multiver...,2022-02-13,60614589,2105964.0,90117.0,UCvC4D8onUfXzvjTOM-dBfEA,Marvel Entertainment,doctor strange in the multiverse of madness,Doctor Strange in the Multiverse of Madness
46,Marvel Studios' Doctor Strange in the Multiver...,2021-12-22,41689494,1705259.0,50557.0,UCvC4D8onUfXzvjTOM-dBfEA,Marvel Entertainment,doctor strange in the multiverse of madness,Doctor Strange in the Multiverse of Madness
...,...,...,...,...,...,...,...,...,...
4824,Barbershop: The Next Cut | Official Trailer 2 ...,2016-01-19,139583,996.0,14.0,UCf5CjDJvsFvtVIhkfmKAwAA,MGM,barbershop: the next cut,Barbershop: The Next Cut
4825,Barbershop: The Next Cut | Official Trailer [HD],2015-12-03,10504,43.0,1.0,UCf5CjDJvsFvtVIhkfmKAwAA,MGM,barbershop: the next cut,Barbershop: The Next Cut
4826,How To Be Single | Official Trailer [HD],2015-11-19,23332,126.0,3.0,UCf5CjDJvsFvtVIhkfmKAwAA,MGM,how to be single,How to Be Single
4828,Creed | Official Trailer 2 [HD],2015-09-15,105921,1071.0,16.0,UCf5CjDJvsFvtVIhkfmKAwAA,MGM,creed,Creed


In [30]:
#group stats by the movie name
movie_stats = videos[videos["Movie"] != ""].groupby("Movie")[["Views", "Likes", "Comments"]].sum()

In [32]:
#merge the movie stats and box_office
movie_stats_complete = pd.merge(box_office, movie_stats, how="left", left_on ="Release", right_on = "Movie")

In [43]:
#check the top 10 movies without stats info
movie_stats_complete[(movie_stats_complete["Views"].isna()) & (movie_stats_complete["Rank"] <= 10)]

,Year,Rank,Release,Gross,Theaters,Total Gross,Release Date,Distributor,Re-Release,Views,Likes,Comments
3,2022,4,The Batman,"$369,345,583",4417,"$369,345,583",2022-03-04,Warner Bros.,NaN,NaN,NaN,NaN
4,2022,5,Minions: The Rise of Gru,"$334,576,925",4427,"$334,576,925",2022-07-01,Universal Pictures,NaN,NaN,NaN,NaN
8,2022,10,Elvis,"$136,457,329",3932,"$136,457,329",2022-06-24,Warner Bros.,NaN,NaN,NaN,NaN
172,2021,5,F9: The Fast Saga,"$173,005,945",4203,"$173,005,945",2021-06-25,Universal Pictures,NaN,NaN,NaN,NaN
174,2021,7,No Time to Die,"$160,772,007",4407,"$160,891,007",2021-10-08,Metro-Goldwyn-Mayer (MGM),NaN,NaN,NaN,NaN
486,2019,4,Frozen II,"$430,144,682",4440,"$477,373,578",2019-11-22,Walt Disney Studios Motion Pictures,NaN,NaN,NaN,NaN
488,2019,6,Star Wars: Episode IX - The Rise of Skywalker,"$390,706,234",4406,"$515,202,542",2019-12-20,Walt Disney Studios Motion Pictures,NaN,NaN,NaN,NaN
662,2018,3,Incredibles 2,"$608,581,744",4410,"$608,581,744",2018-06-15,Walt Disney Studios Motion Pictures,NaN,NaN,NaN,NaN
664,2018,5,Deadpool 2,"$318,491,426",4349,"$318,491,426",2018-05-18,Twentieth Century Fox,NaN,NaN,NaN,NaN
665,2018,6,The Grinch,"$266,280,410",4141,"$270,620,950",2018-11-09,Universal Pictures,NaN,NaN,NaN,NaN


In [47]:
#remove NAs
movie_stats_complete = movie_stats_complete.dropna(subset=["Views"])

In [49]:
# Saving the new dataset in a file
movie_stats_complete.to_excel('movies_stats.xlsx', index=False)

# End